In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import *

from pyomo.environ import *

### Comparing stochastic optimization techniques

We wish to compare solutions to scenario-based optimization problems in the context of renewable resource planning.

##### Setting: 

Hourly CF of wind $w$ is random according to some distribution $F_w: [0, 1] \to [0,1]$. Hourly load $l$ is constant. $F_w$ is not known explicitly but we can draw i.i.d. samples from $F_w$. 

We wish to select the amount of new wind to build on the system $C$ that minimizes total system cost. We have chosen units such that the levelized fixed cost of investment in new wind build is 1. The variable cost of wind is 0, and the variable cost of existing thermal generation is $k$. Thus whenever load exceeds the amount of available wind generation $C*w_i$, the system variable costs are $k * (l - C*w_i)$.

Each random sample $w_i \sim F_w$ corresponds to a "scenario". The corresponding scenario cost for a fixed value of $C$ is $k * \max \{0, l - C*w_i \}$ (k constant). The cost associated with $C$ itself is just $C$.

There are three optimization formulations that we will compare:

#### 1. Flaw-of-Averages / Naive Expected Value Program

$\min C + k * \max \{0, l - C * E[w]\}$

with finite sample approximation

$\min C + k * \max \{0, l - C * \frac{1}{n} \sum_{i=1}^{n}{w_i}\}$

#### 2. Stochastic Program

$\min C + E[ k * \max \{0, l - C * w \} ]$

with finite sample approximation

$\min C + \frac{1}{n} \sum_{i=1}^{n}{ k * \max \{0, l_i - C*w_i \}}$ 

#### 3. Robust Mini-Max Program

$\min C + \sup \{ k * \max \{0, l - C*w \} : w \in support(F_w) \}$

with finite sample approximation

$\min C + \max \{ k * \max \{0, l_i - C*w_i \} : i = 1,..., n\}$

We linearize the term in the objective $\max \{0, l - C*w_i\}$ by replacing it with an auxiliary variable $\eta_i \geq 0$ and adding a constraint $\eta_i \geq l - C*w_i$.

### 0. Collecting random sample of $w_i$

In [126]:
n = 10 # Number of samples to draw
w = 0.8*np.random.rand(n) + 0.2 # Wind CF are uniform(0.2, 0.8)

k = 5 # Variable cost of existing firm generation
l = 100 # MW ; constant level of load

### 1. Flaw-of-Averages / Naive Expected Value Program

In [127]:
# Define sample average of wind CF
w_bar = np.mean(w)

In [128]:
# Construct optimization model

model = ConcreteModel()

# Variables
model.C = Var(domain=NonNegativeReals) # Decision variable for wind build
model.eta_bar = Var(domain=NonNegativeReals) # Auxiliary variable for variable costs

# Parameters
model.k = Param(initialize = k) # Define variable cost of existing firm generation
model.l = Param(initialize = l) # Define constant load level
model.w_bar = Param(initialize = w_bar) # Define sample average of wind CF

# Objective
def objective(m): # Define objective function
    return m.C + m.k*m.eta_bar

model.obj = Objective(rule = objective, sense = minimize)

# Constraints
def const(m):
    return m.eta_bar >= m.l - m.C*m.w_bar

model.const = Constraint(rule = const)


# Solve model
opt = SolverFactory('glpk')
opt.solve(model)

# Print out solution
C_opt = model.C.value
print('Optimal wind build: {:.2f} MW'.format(C_opt))

Optimal wind build: 173.51 MW


### 2. Stochastic Program

In [129]:
# Construct optimization model

model = ConcreteModel()

# Sets
model.samples = RangeSet(n) # Set to define samples

# Variables
model.C = Var(domain=NonNegativeReals) # Decision variable for wind build
model.eta = Var(model.samples, domain=NonNegativeReals) # Auxiliary variables for scenario costs

# Parameters
model.k = Param(initialize = k) # Define variable cost of existing firm generation
model.l = Param(initialize = l) # Define constant load level
model.n = Param(initialize = n) # Define number of samples
model.w = Param(model.samples, initialize = {i+1: w[i] for i in range(n)}) # Define sample average of wind CF

# Objective
def objective(m): # Define objective function
    return m.C + m.k*(1.0/m.n)*sum(m.eta[s] for s in model.samples)

model.obj = Objective(rule = objective, sense = minimize)

# Constraints
def const(m, s):
    return m.eta[s] >= m.l - m.C*m.w[s]

model.const = Constraint(model.samples, rule = const)


# Solve model
opt = SolverFactory('glpk')
opt.solve(model)

# Print out solution
C_opt = model.C.value
print('Optimal wind build: {:.2f} MW'.format(C_opt))

Optimal wind build: 208.27 MW


### 3. Robust Mini-Max Program

In [130]:
# Construct optimization model

model = ConcreteModel()

# Sets
model.samples = RangeSet(n) # Set to define samples

# Variables
model.C = Var(domain=NonNegativeReals) # Decision variable for wind build
model.eta = Var(model.samples, domain=NonNegativeReals) # Auxiliary variables for scenario costs
model.zeta = Var(domain = NonNegativeReals) # Auxiliary variable for maximum of scenario costs

# Parameters
model.k = Param(initialize = k) # Define variable cost of existing firm generation
model.l = Param(initialize = l) # Define constant load level
model.n = Param(initialize = n) # Define number of samples
model.w = Param(model.samples, initialize = {i+1: w[i] for i in range(n)}) # Define sample average of wind CF

# Objective
def objective(m): # Define objective function
    return m.C + m.k*m.zeta

model.obj = Objective(rule = objective, sense = minimize)

# Constraints
def const(m, s):
    return m.eta[s] >= m.l - m.C*m.w[s]

model.const = Constraint(model.samples, rule = const)

def aux_const(m, s):
    return m.zeta >= m.eta[s]

model.aux_const = Constraint(model.samples, rule = aux_const)


# Solve model
opt = SolverFactory('glpk')
opt.solve(model)

# Print out solution
C_opt = model.C.value
print('Optimal wind build: {:.2f} MW'.format(C_opt))

Optimal wind build: 349.88 MW
